# Retrieving pid4cat metadata from the hdl.net API with Python

Details for the endpoints provided by the API can be found in the [auto-generated documentation](https://api.nfdi4cat.org/testpid/v2/index.html).

In [29]:
import httpx
import logging
from pid4cat_model.handle_api import (
    HandleConfig,
    HandleNetAPI,
    HandleAPIRecord,
    pid4cat_record_factory,
)

In [30]:
logger = logging.getLogger(__name__)
# Set logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
logging.basicConfig(level=logging.INFO)

Setup a httpx-client (no authorisation required) and create the default configuration for accessing the pid4cat handle service.

In [31]:
hdl_api_client = httpx.Client()
config = HandleConfig()
config

HandleConfig(api_url='https://hdl.handle.net/api/handles/', prefix='21.T11978', ns_suffix='test', base_url='https://hdl.handle.net/api/handles/21.T11978/test/')

In addition to the config, we need to specify the local_id part for pid4cat handles:

In [32]:
local_id = "KBKA-BEM9"

Now we are ready to request the metadata for this handle identifier (21.T11978/test/KBKA-BME9).

In [33]:
api = HandleNetAPI(config, hdl_api_client)
metadata = api.get_metadata_for_id(local_id)

INFO:httpx:HTTP Request: GET https://hdl.handle.net/api/handles/21.T11978/test/KBKA-BEM9 "HTTP/1.1 200 OK"


For ease of use (and validation) we create a Pid4CatRecord instance via a factory function.

In [34]:
pid_rec = pid4cat_record_factory(metadata)

All metadate elements are easily accessible as attributes of the instance.

In [35]:
available = ", ".join(pid_rec.dict().keys())
print(f"All attrtibutes : {available}")
print(f"Landing page URL: {pid_rec.landing_page_url}")
print(f"Status          : {pid_rec.status}")
print(f"Curation contact: {pid_rec.curation_contact}")

All attrtibutes : landing_page_url, status, schema_version, metadata_license, curation_contact, resource_info, related_identifiers, change_log
Landing page URL: http://Mytest2.com
Status          : REGISTERED
Curation contact: xzy@abc.com


We may also instantiate other classes from the generated pydantic data model, for example an instance of HandleAPIRecord, which closely matches the data model of the handle system.

However, for applications the above Pid4CatRecord instances are much more useful.

In [36]:
print("\nTrying to create a HandleAPIRecord object", end="... ")
hdl_rec = HandleAPIRecord.model_validate(metadata)
print("success!")
print(hdl_rec.model_dump_json(indent=2))


Trying to create a HandleAPIRecord object... success!
{
  "responseCode": 1,
  "handle": "21.T11978/test/KBKA-BEM9",
  "values": [
    {
      "timestamp": "2025-03-11T09:43:14Z",
      "ttl": 86400,
      "type": "URL",
      "index": 1,
      "data": {
        "format": "string",
        "value": "http://Mytest2.com"
      }
    },
    {
      "timestamp": "2025-03-31T08:05:02Z",
      "ttl": 86400,
      "type": "EMAIL",
      "index": 10,
      "data": {
        "format": "string",
        "value": "xzy@abc.com"
      }
    },
    {
      "timestamp": "2025-03-31T08:06:05Z",
      "ttl": 86400,
      "type": "STATUS",
      "index": 11,
      "data": {
        "format": "string",
        "value": "REGISTERED"
      }
    },
    {
      "timestamp": "2025-03-11T10:34:03Z",
      "ttl": 86400,
      "type": "SCHEMA_VER",
      "index": 12,
      "data": {
        "format": "string",
        "value": "v1.0.1"
      }
    },
    {
      "timestamp": "2025-03-11T09:45:25Z",
      "ttl"